For this project, I'm going to look at the correlation between residential property prices and costs for building and construction materials.

https://data.gov.ie/dataset/hpm09-residential-property-price-index   

https://data.gov.ie/dataset/wpm28-wholesale-price-index-excl-vat-for-building-and-construction-materials

I will use CSV files - the reason is that they are the simplest to work with

HPM09.20251124175125.csv - Residential_property_price_index
https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/HPM09/CSV/1.0/en
I've renamed it to - RPPI.csv



WPM28.20251124175238.csv - Wholesale Price Index (Excl VAT) for Building and Construction Materials 
https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/WPM28/CSV/1.0/en
I renamed it to - WPI_BCM.csv


Gathering the Data.
RPPI -
    - It has monthly data from February 2005 to Sept 2025, 
      but to be able to regress against WPI&BCM 
      I will run Sept 2014 to Sept 2014
    - I will remove all the months February 2005 to August 2014 (or not select to import them)

It also has many categories, so i will filter for National - all residential properties
I will be taking index data
I will also ignore all the data talking about the % Change over 12 months for Residential Property Price Index.

WPI-BCM
only goes up to 202409 from 201501
so will probably do Sept 14 to Sept 24

I will filter for the entry of Materials, ignoring the figures for every other category

Cleaning the Data
I will look use the ISNA function to check and replace that i have figures for every entry

Plotting the data, line charts, or indeed regression, 
this should show the posiitve relationship between both sets of data.
Explain, how i could not just pick out any one material because i don't know the proportion that goes into the house, though I could guess.

Investigate - there might be a way of seeing if there is a lagging trend.
Time series data, so need to get the axis working correctly.


In [2]:
# I will want pandas to manipulate my data
import pandas as pd 
# I will want numpy for calculating the descriptive statistics
import numpy as np

# Reading in the Data
We will read in the data in a the form of CSV files from data.gov.ie

In [70]:
# The data is sourced from the Central Statistics Office (CSO) of Ireland
# The dataset is titled "Population by Age Group and Sex"
# The data is available in CSV format via the CSO API
url_rppi = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/HPM09/CSV/1.0/en"
url_wpi_bcm = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/WPM28/CSV/1.0/en"


# read in the data from the csv files into two pandas DataFrames
names_rppi = ["STATISTIC", "Statistic Label", "TLIST(M1)", "Month", "C02803V03373","Type of Residential Property","UNIT", "VALUE"]
df_rppi = pd.read_csv(url_rppi,header=0,usecols=names_rppi)

# We have many rows that we don't need, rows that contain "Percentage Change over 1,3, and 12 months for Residential Property Price Index"
# so let's remove them using regular expressions
df_rppi = df_rppi[df_rppi["Statistic Label"] == "Residential Property Price Index"]
df_rppi = df_rppi[df_rppi["C02803V03373"] == "-"]


number_to_append = "01"
df_rppi["TLIST(M1)"] = df_rppi["TLIST(M1)"].astype("str")
df_rppi["TLIST(M1)"]= df_rppi["TLIST(M1)"]+"01"
df_rppi["TLIST(M1)"] = df_rppi["TLIST(M1)"].astype("datetime64[ns]")



# now that i have many of rows that i don't need removed, I can drop the columns that are no longer necessary
cols_to_drop=  ["STATISTIC", "Month", "Statistic Label", "C02803V03373","Type of Residential Property","UNIT"]
df_rppi = df_rppi.drop(columns=cols_to_drop)

names=["Date","Value"]
df_rppi.columns = names

df_rppi.index = pd.DatetimeIndex(df_rppi["Date"])
df_rppi = df_rppi.drop(columns="Date")
#df_wpi_bcm = pd.read_csv(url_wpi_bcm)
df_rppi.info()
# lets take a look at the first few rows of the DataFrame
print(f"{df_rppi.tail()}\n")
#print(f"{df_wpi_bcm.head()}\n")

df_rppi = df_rppi.loc[df_rppi.index > '2014-08-01']
df_rppi = df_rppi.loc[df_rppi.index < '2024-10-01']

df_rppi.to_csv("rppi_data.csv")



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 249 entries, 2005-01-01 to 2025-09-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   249 non-null    float64
dtypes: float64(1)
memory usage: 3.9 KB
            Value
Date             
2025-05-01  194.0
2025-06-01  196.1
2025-07-01  197.7
2025-08-01  199.5
2025-09-01  201.2



# Add headers - I have no headers
RPPI 
- the first column is just the internal CSO code, add to list of cols to be removed
- second is the index name "Residential Property Price Index", I will change it to RPPI
- third is the year month, I want that column
- fouth is the year month where Month is written alphabetical, maybe want column instead of third
- fifth is the code for the regions don't need, add to list of cols to be removed
- sixth is the region i will filter for all properties i want this
- seventh is base index, i don't need this, add to list of cols to be removed
- eighth is the index, i want this - remember I am starting at row 2299,
- I have another problem, I'm starting at index 95.6 in Sept 2014 because the base is 2015,
but should be fine.



# Cleaning/Simplifying the data
While the data might be clean, there may be a lot of superfluous information in it that we can remove to simplify things.

In [ ]:
# The rppi has many categories, I only want the national and all residential properties, so i will filter for this 
# so let's filter the DataFrame to only include rows 
# where the category column is equal to "Ireland"
# We set our dataframe "df" equal that contains a dataframe comprising of data where for every row, 
# the entry for administrative county is Ireland
df=df[df["Administrative Counties"]== "National - all residential properties"]
df.to_csv("gender_population_for_analysis.csv")
